In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import data_preperation as prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#example of how to call preparation.py
import data_process as proc
import data_preperation as prep
from utils_scenario import *

from utils_baselines import *


#call data_preperation.py 
measurement=prep.prepare_measurement()
forecast = prep.prepare_forecast()

#keep useful columns 
measurement= measurement[['speed', 'cos_wind_dir', 'sin_wind_dir', 'temp', 'radiation', 'precip', 'season', 'am']]


#call data_process.py

steps_in = 48
steps_out = 24


x_df, y_df, x, y_speed = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'speed')
_, _, _, y_cos = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'cos_wind_dir')
_, _, _, y_sin = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'sin_wind_dir')
y_scenarios = get_all_scenarios(y_speed, y_cos, y_sin, b_scenarios = True)
y_dangerous = get_all_dangerous_scenarios(y_speed, y_cos, y_sin)
X_train, X_test, y_train_dangerous, y_test_dangerous = train_test_split(x, y_dangerous, test_size=0.2, shuffle = False)
_, _, y_train_scenarios, y_test_scenarios = train_test_split(x, y_scenarios, test_size=0.2, shuffle = False)
_, _, y_train_speed, y_test_speed = train_test_split(x, y_speed, test_size=0.2, shuffle = False)
_, _, y_train_cos, y_test_cos = train_test_split(x, y_cos, test_size=0.2, shuffle = False)
_, _, y_train_sin, y_test_sin = train_test_split(x, y_sin, test_size=0.2, shuffle = False)

read csv semester csv files from 2015s2 to 2020s1
smooth wind direction
generate seasonality categorical feature
generate am/pm categorical feature
reading forecast data
smooth wind direction


In [ ]:
from julia import Julia
Julia(sysimage='../sys.so')
from interpretableai import iai

In [2]:
names = list(x_df.columns)
names.remove('present_time')
names.remove('forecast_time')
X_train2 = pd.DataFrame(X_train)
X_train2.columns = names

In [ ]:
grid = iai.GridSearch(
    iai.OptimalTreeClassifier(
        random_seed=1,
    ),
    max_depth=range(3, 7),
)
#grid.fit(X_train2, y_train_scenarios)
grid.fit(X_train2, y_train_dangerous)

In [ ]:
grid = iai.GridSearch(
    iai.OptimalTreeClassifier(
        random_seed=1,
    ),
    max_depth=range(5, 6),
)

In [ ]:
#stepsin 48, stepsout 24
lnr = iai.OptimalTreeClassifier(random_seed=1, max_depth = 5, cp = 0.000950473)

In [ ]:
(train_X, train_y), (test_X, test_y) = iai.split_data('regression', X_train, y_train_speed, train_proportion=0.99)
grid = iai.GridSearch(
    iai.OptimalTreeRegressor(
        random_seed=1,
    ),
    max_depth=range(4, 7),
)
grid.fit(train_X, train_y)


In [ ]:
IAI.write_html("tree.html", lnr)

In [ ]:
grid = iai.GridSearch(
    iai.OptimalTreeClassifier(
        random_seed=1,
    ),
    max_depth=range(4, 5),
)